<a href="https://colab.research.google.com/github/tarakantaacharya/TTS_ODIA/blob/main/training_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! pip install datasets

In [ ]:
import pandas as pd

In [ ]:
new_preprocess_data = pd.read_json('/content/new_preprocess_data.json')

In [ ]:
new_preprocess_data.columns

In [ ]:
new_preprocess_data.head()

In [ ]:
from sklearn.model_selection import train_test_split

# Split the DataFrame (90% train, 10% test)
train_df, test_df = train_test_split(new_preprocess_data, test_size=0.1, random_state=42)

# Check the sizes
print(f"Train Data: {len(train_df)} rows")
print(f"Test Data: {len(test_df)} rows")

Train Data: 720 rows
Test Data: 80 rows


In [ ]:
import pandas as pd
from datasets import Dataset

# Convert pandas DataFrame to Hugging Face Dataset
dataset_train = Dataset.from_pandas(train_df)
dataset_test = Dataset.from_pandas(test_df)

# Check if conversion was successful
print(dataset_train)
print(dataset_test)

Dataset({
    features: ['padded_audio', 'padded_text', '__index_level_0__'],
    num_rows: 720
})
Dataset({
    features: ['padded_audio', 'padded_text', '__index_level_0__'],
    num_rows: 80
})


In [ ]:
from transformers import AutoModelForTextToWaveform
import torch

# Load model in int8 quantization without bitsandbytes (works on CPU)
model = AutoModelForTextToWaveform.from_pretrained(
    "facebook/mms-tts-ory",
    torch_dtype=torch.float16,  # Use int8 quantization (CPU-compatible)
    device_map="cpu"  # Force CPU usage
)

print("Model loaded with int8 quantization on CPU successfully!")

config.json:   0%|          | 0.00/1.64k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/145M [00:00<?, ?B/s]

Model loaded with int8 quantization on CPU successfully!


In [ ]:
from transformers import Trainer, TrainingArguments

# Define a function to rename and select the desired columns
def preprocess_function(examples):
    return {
        "input_ids": examples["padded_text"],  # Rename 'padded_text' to 'input_ids'
        "labels": examples["padded_audio"]  # Rename 'padded_audio' to 'labels'
    }

# Apply the preprocessing function to your datasets
dataset_train = dataset_train.map(preprocess_function, remove_columns=dataset_train.column_names)
dataset_test = dataset_test.map(preprocess_function, remove_columns=dataset_test.column_names)


# Training arguments with remove_unused_columns set to False
training_args = TrainingArguments(
    output_dir="./results",  # Output directory for model checkpoints
    evaluation_strategy="steps",
    num_train_epochs=3,  # Number of epochs for fine-tuning
    per_device_train_batch_size=4,  # Use smaller batch size for CPU
    per_device_eval_batch_size=8,
    logging_dir="./logs",
    remove_unused_columns=False  # Ignore unused columns in dataset
)

# Trainer setup for fine-tuning
trainer = Trainer(
    model=model,  # Quantized model
    args=training_args,
    train_dataset=dataset_train,  # Your training data
    eval_dataset=dataset_test,  # Your test data
)

# Fine-tune the model
trainer.train()

Map:   0%|          | 0/720 [00:00<?, ? examples/s]

---

---

In [4]:
model

VitsModel(
  (text_encoder): VitsTextEncoder(
    (embed_tokens): Embedding(76, 192)
    (encoder): VitsEncoder(
      (layers): ModuleList(
        (0-5): 6 x VitsEncoderLayer(
          (attention): VitsAttention(
            (k_proj): Linear(in_features=192, out_features=192, bias=True)
            (v_proj): Linear(in_features=192, out_features=192, bias=True)
            (q_proj): Linear(in_features=192, out_features=192, bias=True)
            (out_proj): Linear(in_features=192, out_features=192, bias=True)
          )
          (dropout): Dropout(p=0.1, inplace=False)
          (layer_norm): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
          (feed_forward): VitsFeedForward(
            (conv_1): Conv1d(192, 768, kernel_size=(3,), stride=(1,))
            (conv_2): Conv1d(768, 192, kernel_size=(3,), stride=(1,))
            (dropout): Dropout(p=0.1, inplace=False)
            (act_fn): ReLU()
          )
          (final_layer_norm): LayerNorm((192,), eps=1e-05, eleme